Assignment - 3

Problem Statement : Build CNN Model for Classification Of Flowers

Mounting the drive

In [1]:
from google.colab import drive
drive.mount ('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd/content/drive/MyDrive/IBM-Project

/content/drive/MyDrive/IBM-Project


In [3]:
ls

flowers/  Flowers-Dataset.zip  flowers.h5  output/


1) Unzipping the dataset

In [4]:
! unzip Flowers-Dataset.zip

Archive:  Flowers-Dataset.zip
  inflating: flowers/daisy/100080576_f52e8ee070_n.jpg  
replace flowers/daisy/10140303196_b88d3d6cec.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

2) Image Augmentation

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range =0.2,horizontal_flip=True,vertical_flip = False)

In [6]:
test_datagen = ImageDataGenerator ( rescale = 1. / 255 )

Splitting data into Train, Validation and Test folders

In [7]:
%pip install split-folders
import splitfolders
splitfolders.ratio('/content/drive/MyDrive/IBM-Project/flowers', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Copying files: 4317 files [01:12, 59.16 files/s] 


In [8]:
x_train = train_datagen.flow_from_directory(r"/content/drive/MyDrive/IBM-Project/output/train",target_size =(64,64),class_mode ='categorical',batch_size=24)

Found 3452 images belonging to 5 classes.


In [9]:
x_train.class_indices

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}

In [10]:
xtest = test_datagen.flow_from_directory('/content/drive/MyDrive/IBM-Project/output/test',
                                         target_size=(64,64),
                                         class_mode='categorical',
                                         batch_size=100)

Found 435 images belonging to 5 classes.


3) Create Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

In [15]:
model = Sequential() #Initializing sequential model

4) Adding layers

In [16]:
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3)))  #Convolution layer
model.add(MaxPooling2D(pool_size=(2,2)))  #MaxPooling layer
model.add(Flatten())  #Flatten layer
model.add(Dense(300,activation='relu'))  #Hidden layer 1
model.add(Dense(150,activation='relu'))  #Hidden layer 2
model.add(Dense(75,activation='relu')) #Hidden layer 3
model.add(Dense(5,activation='softmax')) #Output layer

5) Compile the Model

In [17]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 30752)             0         
                                                                 
 dense_3 (Dense)             (None, 300)               9225900   
                                                                 
 dense_4 (Dense)             (None, 150)               45150     
                                                                 
 dense_5 (Dense)             (None, 75)                11325     
                                                      

6) Fit The Model

In [18]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

Loading the validation data

In [19]:
x_valid = train_datagen.flow_from_directory(r"/content/drive/MyDrive/IBM-Project/output/val",target_size =(64,64),class_mode ='categorical',batch_size=24)

Found 430 images belonging to 5 classes.


In [20]:
early_stopping = EarlyStopping(monitor='val_accuracy',
                        patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
                        patience=5,
                        factor=0.5,min_lr=0.00001)

callback = [reduce_lr,early_stopping]

In [21]:
model.fit(x_train,
                    steps_per_epoch=len(x_train),
                    epochs=30,
                    callbacks=callback,
                    validation_data=x_valid,
                    validation_steps=len(x_valid))

Epoch 1/30
144/144 [==============================] - 25s 167ms/step - loss: 1.3190 - accuracy: 0.4409 - val_loss: 1.1465 - val_accuracy: 0.5349 - lr: 0.0010
Epoch 2/30
144/144 [==============================] - 23s 162ms/step - loss: 1.1008 - accuracy: 0.5623 - val_loss: 1.0215 - val_accuracy: 0.6140 - lr: 0.0010
Epoch 3/30
144/144 [==============================] - 23s 162ms/step - loss: 1.0005 - accuracy: 0.5994 - val_loss: 0.9714 - val_accuracy: 0.6186 - lr: 0.0010
Epoch 4/30
144/144 [==============================] - 23s 158ms/step - loss: 0.9235 - accuracy: 0.6359 - val_loss: 0.9658 - val_accuracy: 0.6116 - lr: 0.0010
Epoch 5/30
144/144 [==============================] - 23s 162ms/step - loss: 0.8563 - accuracy: 0.6712 - val_loss: 0.9994 - val_accuracy: 0.6233 - lr: 0.0010
Epoch 6/30
144/144 [==============================] - 23s 158ms/step - loss: 0.8051 - accuracy: 0.6868 - val_loss: 0.9353 - val_accuracy: 0.6488 - lr: 0.0010
Epoch 7/30
144/144 [==============================] 

7) Save the Model

In [22]:
model.save('flowers.h5')

8) Test the Model

In [29]:
from keras.preprocessing import image
import numpy as np
img=image.load_img('/content/drive/MyDrive/IBM-Project/output/test/dandelion/138166590_47c6cb9dd0.jpg',target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred=np.argmax(model.predict(x))
op=['daisy','dandelion','rose','sunflower','tulip']
op[pred]

'dandelion'

In [30]:
img=image.load_img('/content/drive/MyDrive/IBM-Project/output/test/sunflower/10541580714_ff6b171abd_n.jpg',target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred=np.argmax(model.predict(x))
op=['daisy','dandelion','rose','sunflower','tulip']
op[pred]

'sunflower'

In [32]:
a = model.evaluate_generator(xtest, len(xtest))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [34]:
print(f'Error: {a[0]}')
print(f'Accuracy: {a[1]}')

Error: 1.6151995658874512
Accuracy: 0.6206896305084229
